# Setting up an external computer

## Aim

This tutorial walks through the steps that may be required to submit AiiDA jobs to an external computer, including configuring a new profile, setting up the remote computer, configuring the transport layer and creating an external code.

## Setup

For this tutorial we will assume you have:

<ul>
        <li>AiiDA installed locally</li>
        <li>Quantum Espresso installed on the remote computer</li>
</ul>

### Profile

For simplicity, we will use `verdi presto` to set up our profile, which creates a profile and default user, sets up the localhost computer, and configures the database. 

By default, this will use SQLite for the database, and RabbitMQ will be configured if it is discoverable, but it is not required.


For production work, PostgreSQL and RabbitMQ are both strongly recommended, which can be configured either through `verdi presto`, or manually through `verdi profile setup`.

Please refer to the [AiiDA's "Create a Profile" documentation](https://aiida.readthedocs.io/projects/aiida-core/en/stable/installation/guide_complete.html#create-a-profile) for more details.

In [ ]:
! verdi presto -p scarf

### Computer

If you have a SSH config file setup in ~/.ssh/config, you can directly set the transport layer by following the steps in the ["AiiDA Configuration" documentation](https://aiida.readthedocs.io/projects/aiida-core/en/stable/howto/ssh.html#id1).

However, if you would like to setup a computer from scratch you can do the following: 1 We will set up the remote computer, STFC's [SCARF HPC cluster](https://www.scarf.rl.ac.uk) in this example. The values in square brackets should be changed as required for your remote system.

For more information on these settings, refer to the [AiiDA's "Computer Setup" documentation](https://aiida.readthedocs.io/projects/aiida-core/en/stable/howto/run_codes.html#computer-setup).

**Note:** We are loading the module we want in `prepend_text`. As we will be needing Quantum Espresso to load the code in the later steps we will load it now, otherwise you can leave it empty (`prepend_text: ''`)

In [ ]:
%%writefile ~/computer.yaml
label: [scarf]
description: [https://www.scarf.rl.ac.uk/index.html]
hostname: [ui1.scarf.rl.ac.uk]
transport: core.ssh
scheduler: core.slurm
shebang: '#!/bin/bash'
work_dir: [/work4/scd/scarf/aiida]
mpirun_command: srun -u -n {tot_num_mpiprocs}
mpiprocs_per_machine: 32
use_double_quotes: False
prepend_text: |
  module load amd-modules
  module load QuantumESPRESSO/7.2-foss-2023a
append_text: ''

Run the following commands. First we set up the computer.

In [ ]:
! verdi computer setup -n --config ~/computer.yaml

Then we can configure the computer for the ssh transport, make sure to change the settings in square brackets with your settings.

For more detail, [AiiDA's "Computer Connection Configuration" documentation](https://aiida.readthedocs.io/projects/aiida-core/en/stable/howto/run_codes.html#computer-connection-configuration) goes over how to configure a computer in more detail.

In [ ]:
! verdi computer configure core.ssh [scarf] -n --username [scarf] --key-filename ['/path/to/ssh/key'] --key-policy AutoAddPolicy --safe-interval 30.0

Once the computer is set up, you can test the connection

In [ ]:
! verdi computer test [scarf]

### External Code

If you want to run a calculation that is only available on the external machine, you can set up a code which accesses the external library. For more information, use `verdi code create -h`, or refer to the [AiiDA "Create a Code" documentation](https://aiida.readthedocs.io/projects/aiida-core/en/stable/howto/run_codes.html#how-to-create-a-code)

In [ ]:
%%writefile ~/qe_code.yaml
label: qe
description: Quantum ESPRESSO pw.x
default_calc_job_plugin: quantumespresso.pw
computer: scarf
filepath_executable: /work4/scd/scarf562/eb-amd/software/QuantumESPRESSO/7.2-foss-2023a/bin/pw.x
use_double_quotes: true
prepend_text: ''
append_text: ''

In [ ]:
! verdi code create core.code.installed -n --config ~/qe_code.yaml